In [11]:
import itertools
import time
import pickle

In [74]:
def h_rel(m):
    if m % 4 == 2:
        m //= 2
    if m == 1:
        return 1
    #https://doc.sagemath.org/html/en/reference/modmisc/sage/modular/dirichlet.html
    #TODO: use better algorithm
    if len(m.factor()) == 1:
        Q = 1
    else:
        Q = 2
    DG = DirichletGroup(m)
    ans = Q * (2 * m)
    if m%2 == 0:
        ans /= 2
    for chi in DG:
        chi2 = chi.restrict(chi.conductor())
        if chi2.is_odd():
            ans *= -chi2.bernoulli(1)/2
    return ZZ(ans)

def h_rel_p(m,p):
    #valuations suck !
    if m % 4 == 2:
        m //= 2
    if m == 1:
        return 0
    f = -1
    v = ZZ.valuation(p)
    ans = 0
    if len(m.factor()) != 1:
        ans += v(2)
    DG = DirichletGroup(m)
    ans += v(2) + v(m)
    if m%2 == 0:
        ans -= v(2)
    base_field = -1
    for chi in DG:
        chi2 = chi.restrict(chi.conductor())
        if chi2.is_odd():
            x = -chi2.bernoulli(1)/2
            if f == -1:
                f = x.parent().degree()
            top, bot = (x).norm().as_integer_ratio()
            #print(ans,v(top),v(bot))
            ans += (v(top)-v(bot))/f
    return ans

def h_rel_pm(m,p):
    """
    returns if p divides h_{mp}^-
    """
    if gcd(m,p) > 1 or p == 2: #bad behavior might happen
        return h_rel_p(m*p,p) > 0
    # p is odd, doesn't divide Q, only divides w once
    DG = DirichletGroup(m*p)
    for chi in DG:
        chi2 = chi.restrict(chi.conductor())
        if chi2.is_odd():
            x = -chi2.bernoulli(1)/2
            top = (x).norm().as_integer_ratio()[0]
            if top%p == 0:
                return True
    return False
def h_rel_pm_avoid(m,p,k_set):
    """
    returns if p divides h_{mp}^-, avoids checking through the kp dirichlet subgroup.
    """
    if gcd(m,p) > 1 or p == 2: #bad behavior might happen
        return h_rel_p(m*p,p) > 0
    # p is odd, doesn't divide Q, only divides w once
    DG = DirichletGroup(m*p)
    for chi in DG:
        f = chi.conductor()
        for k in k_set:
            if p*k % f == 0:
                continue
        chi2 = chi.restrict(f)
        if chi2.is_odd():
            x = -chi2.bernoulli(1)/2
            top = (x).norm().as_integer_ratio()[0]
            if top%p == 0:
                return True
    return False

In [3]:
print(h_rel_pm(19,11))
print(h_rel_pm(12,23))
print(h_rel_pm(4,23))
print(h_rel_pm(49,43))

True
True
False
True


In [30]:
print(h_rel_pm(1,11))
print(h_rel_pm_avoid(19,11,[1]))
print(h_rel_pm(19,11))

False
True
True


In [34]:
print(h_rel_pm(1,11))
print(h_rel_pm(2,11))
print(h_rel_pm(3,11))
print(h_rel_pm(6,11))
print(h_rel_pm_avoid(6,11,[1,2,3]))

False
False
False
False
False


In [4]:
h_rel_p(19*11,11)

1

In [5]:
h_rel(19*11)%11

0

In [86]:
# k_irreg[k] gives set of primes that are C_k^- irregular
#i.e. if p divides h_pk, p is in k_irreg[k]
k_irreg = {}

In [87]:
for i in range(1,51):
    if i % 4 != 2:
        k_irreg[i] = set()

In [88]:
def is_irreg(p):
    if p == 2:
        return False
    for B in bernoulli_mod_p(p):
        if B%p == 0:
            return True
    return False

In [89]:
P = Primes()
for i in range(100):
    p = P.unrank(i)
    if is_irreg(p):
        k_irreg[1].add(p)


In [90]:
print(k_irreg)

{1: {257, 131, 389, 263, 523, 271, 401, 149, 409, 283, 157, 541, 37, 293, 421, 433, 307, 311, 59, 67, 461, 463, 467, 347, 353, 101, 103, 233, 491, 379}, 3: set(), 4: set(), 5: set(), 7: set(), 8: set(), 9: set(), 11: set(), 12: set(), 13: set(), 15: set(), 16: set(), 17: set(), 19: set(), 20: set(), 21: set(), 23: set(), 24: set(), 25: set(), 27: set(), 28: set(), 29: set(), 31: set(), 32: set(), 33: set(), 35: set(), 36: set(), 37: set(), 39: set(), 40: set(), 41: set(), 43: set(), 44: set(), 45: set(), 47: set(), 48: set(), 49: set()}


In [91]:
w = list(k_irreg[1])
w.sort()
print(w)

[37, 59, 67, 101, 103, 131, 149, 157, 233, 257, 263, 271, 283, 293, 307, 311, 347, 353, 379, 389, 401, 409, 421, 433, 461, 463, 467, 491, 523, 541]


In [92]:
def update():
    keys = list(k_irreg.keys())
    keys.sort()
    for i in keys:
        return

In [51]:
for i in range(1,37):
    print(i)
    assert(h_rel(37*i)%37 == 0)

1
Case 1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


In [100]:
k = 3
#kirregs = set()
kirregs = {257, 131, 389, 263, 139, 271, 401, 149, 23, 281, 
           409, 283, 157, 419, 37, 293, 421, 47, 433, 307, 
           53, 311, 59, 317, 67, 197, 199, 457, 331, 461, 
           463, 337, 467, 347, 353, 101, 103, 233, 491, 
           113, 241, 379}
#kregs = set()
kregs = {2, 3, 5, 7, 383, 137, 11, 13, 269, 397, 17, 19, 
         277, 151, 29, 31, 163, 167, 41, 43, 173, 431, 
         179, 181, 439, 313, 443, 61, 191, 193, 449, 71, 
         73, 79, 83, 211, 89, 349, 223, 479, 97, 227, 229, 
         359, 487, 107, 109, 239, 367, 499, 373, 503, 251, 509, 127}
ktot = set()
end = 100
autosave_int = 90
time1 = time.time()
for i in range(end):
    p = P.unrank(i)
    if p in kirregs or p in kregs: #already computed
        continue
    if p in k_irreg[1]:
        if k % p != 0: # p is irreg, k != 0 mod p, so p is automatically k-irreg
            kirregs.add(p)
            continue
    if h_rel_pm_avoid(k,p,[1]): # already know it is not irreg, so can avoid speed up a bit
        kirregs.add(p)
    else:
        kregs.add(p)    
    ktot.add(p)
    time2 = time.time()
    if time2 - time1 > autosave_int:
        time1 = time2
        print(kirregs)
        print(kregs)
k_irreg[3] = kirregs
print(kirregs)
print(kregs)

{257, 131, 389, 263, 521, 139, 523, 271, 401, 149, 23, 409, 281, 283, 157, 541, 419, 37, 293, 421, 47, 433, 307, 53, 311, 59, 317, 67, 197, 199, 457, 331, 461, 463, 337, 467, 347, 353, 101, 103, 233, 491, 113, 241, 379}
{2, 3, 5, 7, 137, 11, 13, 269, 397, 17, 19, 277, 151, 29, 31, 163, 167, 41, 43, 173, 431, 179, 181, 439, 313, 443, 61, 191, 193, 449, 71, 73, 79, 83, 211, 89, 349, 223, 479, 97, 227, 229, 359, 487, 107, 109, 239, 367, 499, 373, 503, 251, 127, 509, 383}


In [102]:
k = 5

kirregs = set()
kregs = set()

ktot = set()

end = 100
autosave_int = 90
time1 = time.time()
for i in range(end):
    p = P.unrank(i)
    if p in kirregs or p in kregs: #already computed
        continue
    if p in k_irreg[1]:
        if k % p != 0: # p is irreg, k != 0 mod p, so p is automatically k-irreg
            kirregs.add(p)
            continue
    if h_rel_pm_avoid(k,p,[1]): # already know it is not irreg, so can avoid speed up a bit
        kirregs.add(p)
    else:
        kregs.add(p)    
    ktot.add(p)
    time2 = time.time()
    if time2 - time1 > autosave_int:
        time1 = time2
        print(kirregs)
        print(kregs)
#k_irreg[3] = kirregs
print(kirregs)
print(kregs)

{131, 137, 139, 17, 19, 149, 151, 157, 163, 37, 167, 41, 53, 181, 59, 61, 191, 193, 67, 197, 73, 89, 97, 101, 103, 107, 127}
{2, 3, 5, 7, 11, 13, 23, 29, 31, 43, 173, 47, 179, 71, 199, 79, 83, 211, 223, 227, 109, 113}
{257, 131, 263, 137, 139, 271, 17, 19, 149, 151, 281, 283, 157, 163, 37, 293, 167, 41, 307, 53, 181, 311, 313, 59, 61, 191, 193, 67, 197, 73, 89, 97, 101, 229, 103, 233, 107, 239, 241, 251, 127}
{2, 3, 5, 7, 11, 13, 269, 277, 23, 29, 31, 43, 173, 47, 179, 317, 71, 199, 79, 83, 211, 223, 227, 109, 113}
{257, 131, 263, 137, 139, 271, 17, 19, 149, 151, 281, 283, 157, 163, 37, 293, 167, 41, 307, 53, 181, 311, 313, 59, 61, 191, 193, 67, 197, 73, 331, 337, 89, 347, 349, 97, 353, 101, 229, 103, 233, 107, 239, 241, 251, 127}
{2, 3, 5, 7, 11, 13, 269, 277, 23, 29, 31, 43, 173, 47, 179, 317, 71, 199, 79, 83, 211, 223, 227, 359, 109, 113}
{257, 131, 263, 137, 139, 271, 17, 19, 149, 151, 281, 283, 157, 163, 37, 293, 167, 41, 307, 53, 181, 311, 313, 59, 61, 191, 193, 67, 197, 73, 331,

In [103]:
k = 7

kirregs = set()
kregs = set()

ktot = set()

end = 100
autosave_int = 90
time1 = time.time()
for i in range(end):
    p = P.unrank(i)
    if p in kirregs or p in kregs: #already computed
        continue
    if p in k_irreg[1]:
        if k % p != 0: # p is irreg, k != 0 mod p, so p is automatically k-irreg
            kirregs.add(p)
            continue
    if h_rel_pm_avoid(k,p,[1]): # already know it is not irreg, so can avoid speed up a bit
        kirregs.add(p)
    else:
        kregs.add(p)    
    ktot.add(p)
    time2 = time.time()
    if time2 - time1 > autosave_int:
        time1 = time2
        print(kirregs)
        print(kregs)
#k_irreg[3] = kirregs
print(kirregs)
print(kregs)

{131, 137, 139, 13, 19, 149, 151, 29, 157, 163, 37, 43, 173, 47, 179, 53, 181, 59, 61, 191, 67, 73, 79, 83, 97, 101, 103, 109, 113, 127}
{2, 3, 5, 7, 71, 41, 167, 11, 107, 17, 23, 89, 31}
{131, 137, 139, 13, 19, 149, 151, 29, 157, 163, 37, 43, 173, 47, 179, 53, 181, 59, 61, 191, 193, 67, 199, 73, 79, 83, 211, 223, 97, 101, 103, 109, 113, 127}
{2, 3, 227, 5, 197, 7, 71, 41, 167, 11, 107, 17, 23, 89, 31}
{131, 137, 139, 13, 19, 149, 151, 29, 157, 163, 37, 43, 173, 47, 179, 53, 181, 59, 61, 191, 193, 67, 199, 73, 79, 83, 211, 223, 97, 101, 103, 233, 109, 113, 127}
{2, 3, 227, 5, 197, 7, 71, 41, 167, 11, 107, 229, 239, 17, 23, 89, 31}
{257, 131, 263, 137, 139, 13, 19, 149, 151, 29, 157, 163, 37, 43, 173, 47, 179, 53, 181, 59, 61, 191, 193, 67, 199, 73, 79, 83, 211, 223, 97, 101, 103, 233, 109, 113, 241, 127}
{2, 3, 5, 7, 11, 269, 17, 23, 31, 167, 41, 197, 71, 89, 227, 229, 107, 239, 251}
{257, 131, 389, 263, 383, 137, 139, 13, 397, 271, 401, 19, 149, 277, 151, 281, 409, 283, 29, 157, 163, 

In [104]:
k = 11
kirregs = set()
kregs = set()

ktot = set()

end = 100
autosave_int = 90
time1 = time.time()
for i in range(end):
    p = P.unrank(i)
    if p in kirregs or p in kregs: #already computed
        continue
    if p in k_irreg[1]:
        if k % p != 0: # p is irreg, k != 0 mod p, so p is automatically k-irreg
            kirregs.add(p)
            continue
    if h_rel_pm_avoid(k,p,[1]): # already know it is not irreg, so can avoid speed up a bit
        kirregs.add(p)
    else:
        kregs.add(p)    
    ktot.add(p)
    time2 = time.time()
    if time2 - time1 > autosave_int:
        time1 = time2
        print(kirregs)
        print(kregs)
#k_irreg[3] = kirregs
print(kirregs)
print(kregs)

{67, 37, 5, 71, 101, 41, 73, 43, 103, 107, 79, 17, 89, 59, 61, 31}
{97, 2, 3, 7, 11, 13, 109, 47, 19, 83, 53, 23, 29}
{67, 37, 5, 71, 101, 41, 73, 43, 103, 107, 79, 17, 89, 59, 61, 31}
{97, 2, 3, 7, 11, 13, 109, 47, 113, 19, 83, 53, 23, 29, 127}
{67, 131, 37, 5, 71, 101, 41, 73, 43, 103, 107, 137, 79, 17, 89, 59, 61, 31}
{97, 2, 3, 7, 11, 139, 13, 109, 47, 113, 19, 83, 53, 23, 29, 127}
{131, 5, 137, 17, 149, 151, 157, 31, 163, 37, 41, 43, 59, 61, 67, 71, 73, 79, 89, 101, 103, 107}
{97, 2, 3, 7, 167, 11, 139, 13, 109, 47, 113, 19, 83, 53, 23, 29, 127}
{131, 5, 137, 17, 149, 151, 157, 31, 163, 37, 41, 43, 173, 179, 181, 59, 61, 191, 67, 71, 73, 79, 89, 101, 103, 107}
{97, 2, 3, 193, 7, 167, 11, 139, 13, 109, 47, 113, 19, 83, 53, 23, 29, 127}
{131, 5, 137, 17, 149, 151, 157, 31, 163, 37, 41, 43, 173, 179, 181, 59, 61, 191, 67, 197, 71, 73, 79, 89, 101, 103, 107}
{2, 3, 7, 11, 139, 13, 19, 23, 29, 167, 47, 53, 193, 199, 83, 97, 109, 113, 127}
{131, 5, 137, 17, 149, 151, 157, 31, 163, 37, 4

<ipython-input-104-a9134df28ca2>:18: RuntimeWarning: cypari2 leaked 256032 bytes on the PARI stack
  if h_rel_pm_avoid(k,p,[Integer(1)]): # already know it is not irreg, so can avoid speed up a bit


KeyboardInterrupt: 

In [ ]:
k = 4

kirregs = set()
kregs = set()

ktot = set()

end = 100
autosave_int = 90
time1 = time.time()
for i in range(end):
    p = P.unrank(i)
    if p in kirregs or p in kregs: #already computed
        continue
    if p in k_irreg[1]:
        if k % p != 0: # p is irreg, k != 0 mod p, so p is automatically k-irreg
            kirregs.add(p)
            continue
    if h_rel_pm_avoid(k,p,[1]): # already know it is not irreg, so can avoid speed up a bit
        kirregs.add(p)
    else:
        kregs.add(p)    
    ktot.add(p)
    time2 = time.time()
    if time2 - time1 > autosave_int:
        time1 = time2
        print(kirregs)
        print(kregs)
#k_irreg[3] = kirregs
print(kirregs)
print(kregs)

In [72]:
h_rel(6*23)%23 == 0

True

In [73]:
h_rel(9*23)%23 == 0

True

In [74]:
h_rel(12*23)%23 == 0

True

In [75]:
h_rel(4*23)%23 == 0

False

In [76]:
h_rel(5*23)%23 == 0

False

In [77]:
h_rel(7*23)%23 == 0

False

In [78]:
h_rel(8*23)%23 == 0

True

In [83]:
h_rel(20*23)%23 == 0

False

In [84]:
h_rel(23*23)%23 == 0

Case 1


False

In [41]:
P = Primes()
count = 0
n = 75
for i in range(n):
    p = P.unrank(i)
    if is_irreg(p):
        print(p)
        count += 1
print(n-count)


37
59
67
101
103
131
149
157
233
257
263
271
283
293
307
311
347
353
379
56


In [92]:
h_rel(683*11)%683 == 0

KeyboardInterrupt: 

In [94]:
h_rel(43*49)%43 == 0

True

In [96]:
h_rel(23*13*13)%23 == 0

KeyboardInterrupt: 

In [80]:
print(h_rel_pm(3,2))
print(h_rel_pm(3,3))

False
False


In [73]:
print(h_rel(6))
print(h_rel(9))

1
1


In [79]:
print(h_rel_pm(4,2))
print(h_rel_pm(4,3))

False
False


In [78]:
print(h_rel_pm(5,2))
print(h_rel_pm(5,3))
print(h_rel_pm(5,5))

False
False
False


In [76]:
print(h_rel(10))
print(h_rel(15))
print(h_rel(25))

1
1
1


In [75]:
print(h_rel_pm(7,2))
print(h_rel_pm(7,3))
print(h_rel_pm(7,5))
print(h_rel_pm(7,7))

False
False
False
False


In [52]:
print(h_rel_pm(8,2))
print(h_rel_pm(8,3))
print(h_rel_pm(8,5))
print(h_rel_pm(8,7))

False
False
False
False


False
False
True
False
False


Case 1
1
2
1
1
Case 1
1
2
Case 1
1
2
